The Purpose of this file is to get phrases from the Textual data. This file will take some amount of time to execute. So preferably execute this file on Google Colab.

In [1]:
#Generic Libraries
import pandas as pd
import numpy as np 
import re
#NLP Libraries
import nltk
#Model Libraries
import pickle

In [2]:
df2 = pd.read_csv('data\df_processed_bio.csv')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1057 entries, 0 to 1056
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   bio                 1057 non-null   object
 1   word_tokenized      1057 non-null   object
 2   sentence_tokenized  1057 non-null   object
 3   word_count          1057 non-null   int64 
 4   sentence_count      1057 non-null   int64 
 5   clean_words         1057 non-null   object
 6   clean_stemmed       1057 non-null   object
 7   clean_lemmed        1057 non-null   object
dtypes: int64(2), object(6)
memory usage: 66.2+ KB


In [6]:
# duplicates reduce the df by 651 observations
df2.drop_duplicates('bio', inplace= True)
df2.shape

(1053, 8)

# POS 

In [7]:
def pos_series(keyword):
    '''categorizes after tokenizing words with POS tags'''
    tokens = nltk.word_tokenize(keyword)
    tagged = nltk.pos_tag(tokens)
    return tagged

In [8]:
# cell runs slower due to computational exhaustion; gpu not active
pos_tagged_arrs = df2.bio.apply(pos_series)

In [9]:
# unloads the tuples from the tree object for easier manipulation
pos_tagged = []
for row in pos_tagged_arrs.values:
    for element in row:
        pos_tagged.append(element)

In [10]:
# dataframe contains all of the words with their corresponding pos tag;
pos_df = pd.DataFrame(pos_tagged, columns = ('word','POS'))
# special chars were removed due to irrelevance as a tag but will be included in regex
char_removal = [',', '.', ':', '#', '$', '\'\'', '``', '(', ')','@']
drop_indices = (pos_df.loc[pos_df.POS.isin(char_removal)].index)
pos_df.drop(drop_indices, inplace = True)

# Noun Prase #1

In [11]:
# defining grammer within the text using reg expressions
# optional determinate, any number of adjectives, a noun, noun plural, proper noun with additionals following
grammar1 = ('''Noun Phrases: {<DT>?<JJ>*<NN|NNS|NNP>+}''')
chunkParser = nltk.RegexpParser(grammar1)
tree1 = chunkParser.parse(pos_tagged)

In [12]:
# typical noun phrase pattern to be pickled for later analyses
g1_chunks = []
for subtree in tree1.subtrees(filter=lambda t: t.label() == 'Noun Phrases'):
    # print(subtree)
    g1_chunks.append(subtree)
    

In [14]:
with open('chunks_bio_1.pickle', 'wb') as fp1:
    pickle.dump(g1_chunks, fp1)

# Noun Phrase #2

In [15]:
# Noun phrase variation 
# preposition maybe, any number of adjective or nouns, any plural nouns or singular nouns
grammar2 = ('''NP2: {<IN>?<JJ|NN>*<NNS|NN>} ''')
chunkParser = nltk.RegexpParser(grammar2)
tree2 = chunkParser.parse(pos_tagged)

In [16]:
# variation of a noun phrase pattern to be pickled for later analyses
g2_chunks = []
for subtree in tree2.subtrees(filter=lambda t: t.label() == 'NP2'):
    # print(subtree)
    g2_chunks.append(subtree)
    

In [17]:
with open('chunks_bio_2.pickle', 'wb') as fp2:
    pickle.dump(g2_chunks , fp2)

Verb-Noun Pattern #3

In [18]:
# any sort of verb followed by any number of nouns
grammar3 = ('''
    VS: {<VBG|VBZ|VBP|VBD|VB|VBN><NNS|NN>*}
    ''')
chunkParser = nltk.RegexpParser(grammar3)
tree3 = chunkParser.parse(pos_tagged)

In [19]:
# verb-noun pattern to be pickled for later analyses
g3_chunks = []
for subtree in tree3.subtrees(filter=lambda t: t.label() == 'VS'):
    # print(subtree)
    g3_chunks.append(subtree)

In [20]:
with open('chunks_bio_3.pickle', 'wb') as fp3:
    pickle.dump(g3_chunks, fp3)


# <noun, noun*> Pattern #4

In [21]:
# any number of a singular or plural noun followed by a comma followed by the same noun, noun, noun pattern
grammar4 = ('''
    Commas: {<NN|NNS>*<,><NN|NNS>*<,><NN|NNS>*} 
    ''')
chunkParser = nltk.RegexpParser(grammar4)
tree4 = chunkParser.parse(pos_tagged)

In [22]:
# common pattern of listing skills to be pickled for later analyses
g4_chunks = []
for subtree in tree4.subtrees(filter=lambda t: t.label() == 'Commas'):
    # print(subtree)
    g4_chunks.append(subtree)
    

In [23]:
with open('chunks_bio_4.pickle', 'wb') as fp4:
    pickle.dump(g4_chunks, fp4)